In [5]:
import os

os.environ['LANGCHAIN_API_KEY'] = 'API_key'
os.environ['OPENAI_API_KEY'] = 'API_key'

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'default'

In [12]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())

@traceable(run_type="retriever")
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

@traceable(metadata={"llm": "gpt-3.5-turbo"})
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:

    {docs}""".format(docs='\n'.join(docs))
    return openai_client.chat.completions.create(messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": question},

    ], model="gpt-3.5-turbo")

In [13]:
rag("where did harrison work")

ChatCompletion(id='chatcmpl-9STWkzEuXpReEgmMyCbdUjKpJND04', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Harrison worked at Kensho.', role='assistant', function_call=None, tool_calls=None))], created=1716573714, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=33, total_tokens=40))

In [14]:
import uuid

run_id = str(uuid.uuid4())
rag(
    "where did harrison work",
    langsmith_extra={"run_id": run_id}
)

ChatCompletion(id='chatcmpl-9STWyJb6V0vtc8DeuRv2ZH7yj4Du5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Harrison worked at Kensho.', role='assistant', function_call=None, tool_calls=None))], created=1716573728, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=33, total_tokens=40))

In [15]:
from langsmith import Client
ls_client = Client()

ls_client.create_feedback(
    run_id,
    key="user-score",
    score=1.0,
)

Feedback(id=UUID('997847c3-acaa-4586-9323-cb5b4f5e434b'), created_at=datetime.datetime(2024, 5, 24, 18, 2, 9, 360434, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 5, 24, 18, 2, 9, 360434, tzinfo=datetime.timezone.utc), run_id=UUID('9b4a895d-7d94-4b84-a526-8d3dec54b5e8'), key='user-score', score=1.0, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None)

In [16]:
import uuid

run_id = str(uuid.uuid4())
rag(
    "where did harrison work",
    langsmith_extra={"run_id": run_id, "metadata": {"user_id": "harrison"}}
)

ChatCompletion(id='chatcmpl-9STX2ecOb0UbOwUmfm0c1nDo1dFK7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Harrison worked at Kensho.', role='assistant', function_call=None, tool_calls=None))], created=1716573732, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=33, total_tokens=40))